# Pytest Tutorial - Part III
# Welcome to the third Pytest Talk!

# My name is Jack Camier, Data Scientist and Python Developer
## Found below is a link to the first two talks of this series:
https://github.com/jcamier/pytest-docker/blob/master/projects/pytest_tutorial.ipynb

https://github.com/jcamier/pytest-docker/blob/master/projects/pytest_tutorial_2.ipynb

# Inspiration for this talk is based on Brian Okken's book, "Python Testing with pytest"
https://pragprog.com/book/bopytest/python-testing-with-pytest
# Brian Okken has a great podcast dedicated to testing with python called Test & Code:
https://testandcode.com/
# Also, special thanks to Christopher Prohm who created a package to run pytest in Jupyter Notebooks called ipytest
https://github.com/chmp/ipytest

## This section we will be focusing on fixtures and scope

## First we need to import the ipytest package and its magic methods that allows us to use pytest in a Jupyter notebook

In [1]:
import os
import pytest
import ipytest
import ipytest.magics
# Enable ipython AST transforms to rewrite asserts, defaults to False. Discussed in my last talk why.
ipytest.config.rewrite_asserts = True

# Name of the workbook, so pytest knows where to run the tests

In [2]:
__file__ = "pytest_tutorial_3.ipynb"

In [3]:
YOUR_DIR_FOR_PYTEST_TUTORIAL_3 = "/Users/jacquescamier/DFW_Python_Talks/pytest_series"

# Reminders:
## Test files should be named test_something.py or something_test.py. 
## Test methods and functions should be named test_something. 
## Test classes should be named TestSomething.
----

# Fixtures are functions that are run by pytest before (sometimes after) the actual test functions.

In [4]:
%%HTML
<img src="../images/fixtures.gif" width="400"/>

## "The code in the fixture can do whatever you want it to. You can use fixtures to get a data set for the tests to work on. You can use fixtures to get a system into a known state before running a test. Fixtures are also used to get data ready for multiple tests."

- Okken, Brian. Python Testing with pytest: Simple, Rapid, Effective, and Scalable . Pragmatic Bookshelf. Kindle Edition. 

# The @pytest.fixture() decorator is used to tell pytest that a function is a fixture. When you include the fixture name in the parameter list of a test function, pytest knows to run it before running the test.

# Let's do a quick example

## Use ipytest magic commands to run pytest in jupyter notebook. ipytest.clean_tests() clears any previously defined tests

In [5]:
ipytest.clean_tests()

In [6]:
@pytest.fixture()
def set_data():
    return 50

In [7]:
%%run_pytest -qq

# Create failing test
def test_set_data(set_data):
    assert set_data == 49

F                                                                        [100%]
=================================== FAILURES ===================================
________________________________ test_set_data _________________________________

set_data = 50

    def test_set_data(set_data):
>       assert set_data == 49
E       assert 50 == 49
E         -50
E         +49

<ipython-input-7-e5a6e4e81ad5>:4: AssertionError


In [8]:
%%run_pytest -qq

# Create passing test
def test_set_data(set_data):
    assert set_data == 50

.                                                                        [100%]


# Let's do another fixture example

In [9]:
ipytest.clean_tests()

In [10]:
# Create test fixture of a dictionary function
@pytest.fixture
def dict_list():
    return [
        dict(a='a', b=3),
        dict(a='c', b=1),
        dict(a='b', b=2),
    ]

In [11]:
%%run_pytest -qq

# Pass the fixture function to the test function
def test_sorted__key_example_1(dict_list):
    assert sorted(dict_list, key=lambda d: d['a']) == [
        dict(a='a', b=3),
        dict(a='b', b=2),
        dict(a='c', b=1),
    ]

.                                                                        [100%]


# pytest fixtures are one of the core features that make pytest stand out above other test frameworks

## *side note fixtures in pytest are not the same as fixtures in Django. In Django a fixture normally means to get some initial data loaded into a database at the start of the application.

# You can put fixtures into individual test files, but to share fixtures among multiple test files, then you need to use a conftest.py file.

# For today's talk, I will not be creating a conftest.py since all the tests will be done in this jupyter notebook

# Fixtures include an optional parameter called scope. This controls how often fixtures get set up or torn down.

In [12]:
%%HTML
<img src="../images/scope.png" width="200"/>

# Possible fixture values of scope are: function, class, module, package or session. The default scope is function

# scope='function'
## Run once per test function. The setup portion is run before each test using the fixture. The teardown portion is run after each test using the fixture. This is the default scope used when no scope parameter is specified.

# scope='class'
## Run once per test class, regardless of how many test methods are in the class.

# scope='module'
## Run once per module, regardless of how many test functions or methods or other fixtures in the module use it.

# scope='session'
## Run once per session. All test methods and functions using a fixture of session scope share one setup and teardown call.

- Okken, Brian. Python Testing with pytest: Simple, Rapid, Effective, and Scalable . Pragmatic Bookshelf. Kindle Edition. 

# I created the above mentioned fixture and test in a file called test_dict.py

In [13]:
%%HTML
<img src="../images/test_dict_screenshot.png" width="600"/>

# Using the "pytest --setup-show" command argument, we can see how:
## our test performed
## the setup
## the fixture that is being used
## and the teardown

In [14]:
! pytest --setup-show test_dict.py

============================= test session starts ==============================
platform darwin -- Python 2.7.14, pytest-3.2.1, py-1.4.34, pluggy-0.4.0
rootdir: /Users/jacquescamier/DFW_Python_Talks/pytest_series, inifile:
collected 1 item                                                                

test_dict.py 
      SETUP    F dict_list
        test_dict.py::test_sorted__key_example_1 (fixtures used: dict_list).
      TEARDOWN F dict_list

=========================== 1 passed in 0.01 seconds ===========================


# Let's test a Django web app that is using Reactjs with pytest
## The source code is found here:
https://github.com/jcamier/pytest-django-react

In [15]:
# Change the global DJANGO_APP_DIR var to your directory
DJANGO_APP_DIR = "/Users/jacquescamier/PycharmProjects/django-react-app/leadmanager"
os.chdir(DJANGO_APP_DIR)
os.listdir()

['.babelrc',
 'frontend',
 'leads',
 'node_modules',
 'db.sqlite3',
 'webpack.config.js',
 'README.md',
 '.gitignore',
 'package-lock.json',
 'package.json',
 'links',
 'manage.py',
 '.git',
 'leadmanager']

# Go to Django root directory and run python manage.py runserver
http://127.0.0.1:8000/

# Let's add a lead message in our app

# This was built using Django rest_framework
https://www.django-rest-framework.org/

# Django rest_framework comes with built in REST features
# Let's add a lead using the Django built in POST functionality
## We can see the functionality by going to the link below
http://127.0.0.1:8000/api/leads/

In [16]:
ipytest.clean_tests()

In [17]:
os.chdir(YOUR_DIR_FOR_PYTEST_TUTORIAL_3)

In [19]:
# Let's use requests to do a simple get call
import requests
url = "http://127.0.0.1:8000/api/leads/"
r = requests.get(url)

r.text 

'[{"id":1,"name":"John Doe","email":"jdoe@gmail.com","message":"Please contact John","created_at":"2019-06-04T12:51:50.969056Z"},{"id":2,"name":"Sam Smith","email":"sam@gmail.com","message":"Please contact Bill","created_at":"2019-06-04T12:52:56.896960Z"},{"id":5,"name":"Chris Cram","email":"chris@gmail.com","message":"Chris follow up with Sam","created_at":"2019-06-06T00:44:01.151338Z"},{"id":6,"name":"George TheSalesman","email":"george@gmail.com","message":"George follow up with Chris on a car","created_at":"2019-06-06T12:40:29.124993Z"}]'

In [20]:
ipytest.clean_tests()

In [21]:
# We could make the url a fixture
@pytest.fixture
def assign_url():
    return "http://127.0.0.1:8000/api/leads/"

In [22]:
%%run_pytest -qq
def test_http_response(assign_url):
    assert assign_url == "http://127.0.0.1:8000/api/leads/"

.                                                                        [100%]


# Okay, so that test passed

In [23]:
ipytest.clean_tests()

In [24]:
@pytest.fixture
def assign_url():
    return "http://127.0.0.1:8000/api/leads/"

In [25]:
%%run_pytest -qq
def test_requests_get_fail(assign_url):
    url = assign_url
    r = requests.get(url)
    requests_text = r.text
    assert requests_text is None
    
def test_requests_get(assign_url):
    url = assign_url
    r = requests.get(url)
    requests_text = r.text
    assert requests_text is not None    

F.                                                                       [100%]
=================================== FAILURES ===================================
____________________________ test_requests_get_fail ____________________________

assign_url = 'http://127.0.0.1:8000/api/leads/'

    def test_requests_get_fail(assign_url):
        url = assign_url
        r = requests.get(url)
        requests_text = r.text
>       assert requests_text is None
E       assert '[{"id":1,"name":"John Doe","email":"jdoe@gmail.com","message":"Please contact John","created_at":"2019-06-04T12:51:50...mail":"george@gmail.com","message":"George follow up with Chris on a car","created_at":"2019-06-06T12:40:29.124993Z"}]' is None

<ipython-input-25-09f06fd2429b>:5: AssertionError


In [26]:
ipytest.clean_tests()

In [27]:
%%run_pytest -qq
def test_requests_http_status_code(assign_url):
    url = assign_url
    r = requests.get(url)
    requests_status_code = r.status_code
    assert requests_status_code == 200

.                                                                        [100%]


In [28]:
ipytest.clean_tests()

In [29]:
! pytest --setup-show test_django_requests.py

============================= test session starts ==============================
platform darwin -- Python 2.7.14, pytest-3.2.1, py-1.4.34, pluggy-0.4.0
rootdir: /Users/jacquescamier/DFW_Python_Talks/pytest_series, inifile:
collected 4 items                                                               

test_django_requests.py 
      SETUP    F assign_url
        test_django_requests.py::test_http_response (fixtures used: assign_url).
      TEARDOWN F assign_url
      SETUP    F assign_url
        test_django_requests.py::test_requests_get_fail (fixtures used: assign_url)F
      TEARDOWN F assign_url
      SETUP    F assign_url
        test_django_requests.py::test_requests_get (fixtures used: assign_url).
      TEARDOWN F assign_url
      SETUP    F assign_url
        test_django_requests.py::test_requests_http_status_code (fixtures used: assign_url).
      TEARDOWN F assign_url

=================================== FAILURES ===================================
________________________

# Thank you.